In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cross-border-trade-and-customs-delay-dataset/trade_customs_dataset.csv


**Data cleaning and Preprocessing**

1) Getting an overview of the dataset

In [2]:
df = pd.read_csv("/kaggle/input/cross-border-trade-and-customs-delay-dataset/trade_customs_dataset.csv")
df.head()
df.info()
df.isnull().sum

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Shipment_ID             10000 non-null  object 
 1   Origin_Country          10000 non-null  object 
 2   Destination_Country     10000 non-null  object 
 3   Shipment_Date           10000 non-null  object 
 4   Estimated_Arrival_Date  10000 non-null  object 
 5   Actual_Arrival_Date     10000 non-null  object 
 6   Transport_Mode          10000 non-null  object 
 7   Carrier_Name            10000 non-null  object 
 8   Route_Code              10000 non-null  object 
 9   Commodity_Type          10000 non-null  object 
 10  Declared_Value_USD      10000 non-null  float64
 11  Weight_kg               10000 non-null  float64
 12  HS_Code                 10000 non-null  float64
 13  Document_Status         10000 non-null  object 
 14  Compliance_Score        10000 non-null 

<bound method DataFrame.sum of       Shipment_ID  Origin_Country  Destination_Country  Shipment_Date  \
0           False           False                False          False   
1           False           False                False          False   
2           False           False                False          False   
3           False           False                False          False   
4           False           False                False          False   
...           ...             ...                  ...            ...   
9995        False           False                False          False   
9996        False           False                False          False   
9997        False           False                False          False   
9998        False           False                False          False   
9999        False           False                False          False   

      Estimated_Arrival_Date  Actual_Arrival_Date  Transport_Mode  \
0                      

In [3]:
#converting date columns to the date-time format
date_cols = [
    'Shipment_Date',
    'Estimated_Arrival_Date',
    'Actual_Arrival_Date'
]

for col in date_cols:
    df[col] = pd.to_datetime(df[col])

df[date_cols].dtypes

Shipment_Date             datetime64[ns]
Estimated_Arrival_Date    datetime64[ns]
Actual_Arrival_Date       datetime64[ns]
dtype: object

2) **Temporal feature engineering**
 * Checking if : the transit days are all positive, arrivak delay can be 0 or positive, months between 1-12 and weekday between 0-6

In [4]:
#planned transit time 
df['Planned_Transit_Days'] = (df['Estimated_Arrival_Date'] - df['Shipment_Date']).dt.days
#actual transit time 
df['Actual_Transit_Days'] = (df['Actual_Arrival_Date'] - df['Shipment_Date']).dt.days
#Arrival delay relative to plan 
df['Arrival_Delay_Days'] = (df['Actual_Arrival_Date'] - df['Estimated_Arrival_Date']
).dt.days
#calendar features 
df['Shipment_Month'] = df['Shipment_Date'].dt.month
df['Shipment_Weekday'] = df['Shipment_Date'].dt.weekday

df[
    [
        'Planned_Transit_Days',
        'Actual_Transit_Days',
        'Arrival_Delay_Days',
        'Shipment_Month',
        'Shipment_Weekday'
    ]
].head()

,Planned_Transit_Days,Actual_Transit_Days,Arrival_Delay_Days,Shipment_Month,Shipment_Weekday
0,12,15,3,1,0
1,24,25,1,8,5
2,15,15,0,9,3
3,22,25,3,2,6
4,29,29,0,7,3


3. **Compliance and Offense-Based Risk Features**
   * checking for the values of risk scores and offenses from the dataset

In [5]:
# Prior offense indicator
df['Has_Prior_Offense'] = (df['Prior_Offense_Count'] > 0).astype(int)
# Composite compliance risk score
df['Compliance_Risk_Score'] = (
    (1 - df['Compliance_Score']) +
    (df['Prior_Offense_Count'] * 0.3)
)
# Document issue flag
df['Document_Issue'] = df['Document_Status'].isin(['Missing', 'Error']).astype(int)
# Verify
df[
    [
        'Compliance_Score',
        'Prior_Offense_Count',
        'Has_Prior_Offense',
        'Compliance_Risk_Score',
        'Document_Status',
        'Document_Issue'
    ]
].head()

,Compliance_Score,Prior_Offense_Count,Has_Prior_Offense,Compliance_Risk_Score,Document_Status,Document_Issue
0,0.49,3,1,1.41,Error,1
1,0.76,3,1,1.14,Complete,0
2,0.58,0,0,0.42,Missing,1
3,0.77,2,1,0.83,Error,1
4,0.52,3,1,1.38,Missing,1


4.**Route Risk Bucketing**
* checking if the risk index is between 0 and 1 and risk levels assigned into low/med/high
* this helps to form the target for classification

In [6]:

df['Route_Risk_Level'] = pd.cut(
    df['Route_Risk_Index'],
    bins=[0, 0.33, 0.66, 1.0],
    labels=['Low', 'Medium', 'High']
)

df[['Route_Risk_Index', 'Route_Risk_Level']].head()

,Route_Risk_Index,Route_Risk_Level
0,0.16,Low
1,0.71,High
2,0.52,Medium
3,0.14,Low
4,0.51,Medium


5. **Text Preprocessing - reason for delays**
   * converting text reasons(for delay) from the tables into numeric features
   * TF-IDF captures important words(eg - inspection,document,compliance) and reduces impact of very common words

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    stop_words='english',
    max_features=300
)
delay_tfidf = tfidf.fit_transform(df['Delay_Reason'])
delay_tfidf.shape

(10000, 28)

6. **Encoding Categorical Variables**
   * convert text labels from columns like country names, transport modes, carrier names, commodity types, inspection types into numerical inputs for ml models to make it easier for tree-based models and baseline comparisons
   * avoiding one-hot here to keep dimensionality under control as there are too many columns and this can make the model slower,harder to explain and messier for visualization.

In [8]:
from sklearn.preprocessing import LabelEncoder

categorical_cols = [
    'Origin_Country',
    'Destination_Country',
    'Transport_Mode',
    'Carrier_Name',
    'Route_Code',
    'Commodity_Type',
    'Tariff_Category',
    'Inspection_Type',
    'Document_Status',
    'Route_Risk_Level'
]
le = LabelEncoder()
for col in categorical_cols:
    df[col] = le.fit_transform(df[col])
df[categorical_cols].head()

,Origin_Country,Destination_Country,Transport_Mode,Carrier_Name,Route_Code,Commodity_Type,Tariff_Category,Inspection_Type,Document_Status,Route_Risk_Level
0,4,2,0,49,754,1,1,2,1,1
1,3,1,3,29,203,3,0,1,0,0
2,2,3,2,1,220,0,2,1,2,2
3,1,2,3,17,80,2,0,2,1,1
4,4,2,2,46,875,4,0,1,2,2


7.**Scaling Numeric Features**
* scaling numeric values of varied ranges from the columns to a common scale to prevent large values columns dominate model learning
* StandardScaler centers values around 0, sclaes to unit variance

In [9]:
from sklearn.preprocessing import StandardScaler

numeric_cols = [
    'Declared_Value_USD',
    'Weight_kg',
    'Compliance_Score',
    'Prior_Offense_Count',
    'Route_Risk_Index',
    'Planned_Transit_Days',
    'Actual_Transit_Days'
]
scaler = StandardScaler()
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])
df[numeric_cols].describe().loc[['mean', 'std']]

,Declared_Value_USD,Weight_kg,Compliance_Score,Prior_Offense_Count,Route_Risk_Index,Planned_Transit_Days,Actual_Transit_Days
mean,-3.836931e-17,9.698908e-17,1.513456e-16,4.440892e-17,-7.247536e-17,-1.104894e-16,-8.952838e-17
std,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00


dropping raw columns which wont be needed further

In [10]:
drop_cols = [
    'Shipment_ID',
    'Shipment_Date',
    'Estimated_Arrival_Date',
    'Actual_Arrival_Date',
    'Delay_Reason'
]
df.drop(columns=drop_cols, inplace=True)
df.shape

(10000, 26)

Above cells marks the completion of cleaning and preprocessing so the cells that follow will be of getting the data ready for further steps of the project 

In [11]:
#1) Defining feature groups

# Regression target (delay prediction)
TARGET_DELAY = 'Arrival_Delay_Days'

# Classification target (risk prediction)
TARGET_RISK = 'Route_Risk_Level'

# Numeric features (scaled)
NUMERIC_COLS = [
    'Declared_Value_USD',
    'Weight_kg',
    'Compliance_Score',
    'Prior_Offense_Count',
    'Route_Risk_Index',
    'Planned_Transit_Days',
    'Actual_Transit_Days',
    'Compliance_Risk_Score'
]

# Encoded categorical features
CATEGORICAL_COLS = [
    'Origin_Country',
    'Destination_Country',
    'Transport_Mode',
    'Carrier_Name',
    'Route_Code',
    'Commodity_Type',
    'Tariff_Category',
    'Inspection_Type',
    'Document_Status'
]

print("Numeric features:", len(NUMERIC_COLS))
print("Categorical features:", len(CATEGORICAL_COLS))
print("---------------------------")

#2) Defining modelling inputs and targets 

# Regression target
y_reg = df['Arrival_Delay_Days']  
# Classification target
y_clf = df['Route_Risk_Level'] 

exclude_cols = [
    'Arrival_Delay_Days',
    'Route_Risk_Level',
    'Risk_Flag' 
]

X = df.drop(columns=exclude_cols)

X_reg = X.copy()
X_clf = X.copy()

# sanity check
print("X_reg shape:", X_reg.shape)
print("y_reg shape:", y_reg.shape)
print("X_clf shape:", X_clf.shape)
print("y_clf shape:", y_clf.shape)
print("---------------------------")

#3) explicit feature lists
numeric_features = [
    'Declared_Value_USD',
    'Weight_kg',
    'Compliance_Score',
    'Prior_Offense_Count',
    'Route_Risk_Index',
    'Planned_Transit_Days',
    'Actual_Transit_Days',
    'Compliance_Risk_Score'
]

categorical_features = [
    'Origin_Country',
    'Destination_Country',
    'Transport_Mode',
    'Carrier_Name',
    'Route_Code',
    'Commodity_Type',
    'Tariff_Category',
    'Inspection_Type',
    'Document_Status'
]

#final feature list
model_features = numeric_features + categorical_features

print("Numeric features:", len(numeric_features))
print("Categorical features:", len(categorical_features))
print("Total model features:", len(model_features))


Numeric features: 8
Categorical features: 9
---------------------------
X_reg shape: (10000, 23)
y_reg shape: (10000,)
X_clf shape: (10000, 23)
y_clf shape: (10000,)
---------------------------
Numeric features: 8
Categorical features: 9
Total model features: 17


In [12]:
# getting separate tables ready for further steps
regression_df = X_reg.copy()
regression_df['Arrival_Delay_Days'] = y_reg

classification_df = X_clf.copy()
classification_df['Route_Risk_Level'] = y_clf


regression_df.to_csv('/kaggle/working/customs_regression_dataset.csv', index=False)
classification_df.to_csv('/kaggle/working/customs_classification_dataset.csv', index=False)

# saving feature metadata
feature_metadata = {
    'numeric_features': numeric_features,
    'categorical_features': categorical_features,
    'regression_target': 'Arrival_Delay_Days',
    'classification_target': 'Route_Risk_Level'
}

pd.Series(feature_metadata).to_json(
    '/kaggle/working/feature_metadata.json',
    indent=4
)

print("Files saved successfully.")

Files saved successfully.
